In [1]:
# Read a CSV file with DOIs, resolve them to WIDs using OpenAlex API,
# and fetch citation counts by year for each WID.

import pandas as pd
import requests
import time

# ---- Load CSV and take first 20 rows ----
df = pd.read_csv("nobel_prize_data_ss.csv")
#df_subset = df.head(20)

# ---- Determine WID from DOI column ----
def resolve_wid(value):
    value = str(value).strip()
    if not value or value.lower() == 'nan':
        return None
    if value.upper().startswith('W'):
        return value  # Already a WID
    elif value.startswith('10.'):  # Looks like a DOI
        url = f"https://api.openalex.org/works/doi:{value.lower()}"
        try:
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                return data['id'].split('/')[-1]
            else:
                print(f"❌ DOI lookup failed for {value}, status code {response.status_code}")
        except Exception as e:
            print(f"⚠️ Error resolving DOI {value}: {e}")
    return None

# ---- Fetch citations by year ----
def get_citations_by_year(wid):
    url = "https://api.openalex.org/works"
    params = {
        "filter": f"cites:{wid}",
        "group_by": "publication_year",
        "per_page": 200
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return {entry['key']: entry['count'] for entry in data.get('group_by', [])}
        else:
            print(f"❌ Citation fetch failed for WID {wid}, status {response.status_code}")
    except Exception as e:
        print(f"⚠️ Error fetching citations for {wid}: {e}")
    return {}

# ---- Main loop ----
all_years = set()
results = []

for idx, row in df.iterrows():
    input_id = row['DOI']
    print(f"[{idx+1}] Processing: {input_id}")

    wid = resolve_wid(input_id)
    if not wid:
        print(f"⚠️ Skipping: could not resolve WID for {input_id}")
        continue

    citations = get_citations_by_year(wid)
    all_years.update(citations.keys())

    result_row = {
        'Original_Input': input_id,
        'Resolved_WID': wid
    }
    result_row.update(citations)
    results.append(result_row)

    time.sleep(0.5)  # Respect OpenAlex API rate limit

# ---- Format results ----
all_years_sorted = sorted(all_years)
final_df = pd.DataFrame(results)

# Fill in missing years with 0
for year in all_years_sorted:
    if year not in final_df.columns:
        final_df[year] = 0

# Reorder columns
final_df = final_df[['Original_Input', 'Resolved_WID'] + all_years_sorted]

# ---- Save to file ----
output_path = "openalex_citation_by_year_new.csv"
final_df.to_csv(output_path, index=False)
print(f"✅ Done! Saved to {output_path}")


[1] Processing: nan
⚠️ Skipping: could not resolve WID for nan
[2] Processing: 10.1103/PhysRevLett.9.439
[3] Processing: W1580307541
[4] Processing: 10.1103/PhysRevLett.2.256
[5] Processing: 10.1103/PhysRev.111.747
[6] Processing: 10.1113/jphysiol.1952.sp004759
[7] Processing: 10.1113/jphysiol.1954.sp005226
[8] Processing: 10.1038/248701a0
[9] Processing: 10.1038/251547a0
[10] Processing: 10.1038/248701a0
[11] Processing: 10.1038/251547a0
[12] Processing: 10.1086/300499
[13] Processing: 10.1073/pnas.41.11.849
[14] Processing: W1566766802
[15] Processing: 10.1073/pnas.73.10.3628
[16] Processing: nan
⚠️ Skipping: could not resolve WID for nan
[17] Processing: nan
⚠️ Skipping: could not resolve WID for nan
[18] Processing: 10.1038/226321a0
[19] Processing: 10.1103/PhysRev.30.705
[20] Processing: 10.1007/978-94-009-0657-0_7
[21] Processing: 10.1103/PhysRev.74.230
[22] Processing: 10.1103/PhysRev.80.72
[23] Processing: 10.1002/j.1538-7305.1949.tb03645.x
[24] Processing: 10.1103/PhysRev.109.

In [2]:
# Drop duplicates based on Original_Input and Resolved_WID

import pandas as pd

# Load your file
df = pd.read_csv("openalex_citation_by_year_new.csv")

# Remove exact duplicates
df = df.drop_duplicates()

# Remove duplicates based on DOI and WID
df = df.drop_duplicates(subset=["Original_Input", "Resolved_WID"])

# Save or use as needed
df.to_csv("openalex_citation_by_year_deduplicated.csv", index=False)


In [3]:
# Merge the citation data with the original metadata

import pandas as pd

# ---- Step 1: Load both CSVs ----
original_df = pd.read_csv("nobel_prize_data_ss.csv")  # the one with metadata
citations_df = pd.read_csv("openalex_citation_by_year_deduplicated.csv")  # the one with WID + citation years

# ---- Step 2: Merge on DOI (case-insensitive match) ----
# Change name of the DOI column in original_df to match citations_df
original_df.rename(columns={"DOI": "Original_Input"}, inplace=True)
original_df['Original_Input'] = original_df['Original_Input'].str.lower().str.strip()
citations_df['Original_Input'] = citations_df['Original_Input'].str.lower().str.strip()

merged_df = pd.merge(original_df, citations_df, on="Original_Input", how="left")

# ---- Step 3: Save the merged file ----
merged_df.to_csv("nobel_with_citations_full.csv", index=False)
print("✅ Saved as 'nobel_with_citations_full.csv'")


✅ Saved as 'nobel_with_citations_full.csv'


In [4]:
# Drop spurious columns and clean up the DataFrame

import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("nobel_with_citations_full.csv")

# Identify citation year columns
year_columns = [col for col in df.columns if col.isdigit()]
df[year_columns] = df[year_columns].apply(pd.to_numeric, errors='coerce').fillna(0)

# Create mask to zero out citations before publication year
year_array = np.array([int(y) for y in year_columns])
pub_years = df['Pub year'].values.reshape(-1, 1)
mask = year_array <= pub_years
df.loc[:, year_columns] = np.where(mask, 0, df[year_columns].values)

# Defragment DataFrame before adding column
df = df.copy()

# Add total citations column
df['Total Citations'] = df[year_columns].sum(axis=1)

# Now drop all year columns before the earliest Pub year
min_pub_year = int(df['Pub year'].dropna().min())
columns_to_drop = [col for col in year_columns if int(col) < min_pub_year]
df.drop(columns=columns_to_drop, inplace=True)

# Sort by Prize year
df.sort_values(by='Prize year', inplace=True)

# Optional: Save cleaned file
df.to_csv("nobel_with_citations_cleaned.csv", index=False)
